# Confronto Pratico: Modelli BASE vs INSTRUCT

## Obiettivo
Questo notebook dimostra le differenze pratiche tra modelli **BASE** (pretrained) e **INSTRUCT** (instruction-tuned) attraverso:
- Confronto di tecniche di prompting
- Batteria di 5 task diversi
- Analisi di safety/refusal behavior
- Metriche di performance e accuratezza

**Modelli utilizzati:**
- BASE: Modello pre-addestrato (completamento di testo)
- INSTRUCT: Modello fine-tuned per seguire istruzioni

---


## Setup e Installazione Dipendenze


In [ ]:
# Installazione delle dipendenze necessarie
%pip install transformers>=4.42 accelerate torch datasets evaluate sacrebleu rouge-score tiktoken matplotlib pandas numpy


In [ ]:
# Import delle librerie
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re
import time
import random
from typing import Dict, List, Tuple, Any
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
import warnings
warnings.filterwarnings('ignore')

# Impostazione del seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

print("Setup completato!")
print(f"CUDA disponibile: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")


## Configurazione Modelli


In [ ]:
# ============ CONFIGURAZIONE ============
CONFIG = {
    'provider': 'hf',
    'base_model_id': 'Qwen/Qwen2.5-7B',  # Modello BASE
    'instruct_model_id': 'Qwen/Qwen2.5-7B-Instruct',  # Modello INSTRUCT
    'dtype': 'auto',
    'device_map': 'auto',
    'max_new_tokens': 256,
    'temperature': 0.2,
    'load_in_8bit': False,  # Cambia a True se hai problemi di memoria
    'load_in_4bit': False,  # Alternativa per memoria molto limitata
    'trust_remote_code': False
}

print("Configurazione:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")


## Caricamento Modelli e Tokenizer


In [ ]:
# Caricamento del modello BASE
print("Caricamento modello BASE...")
base_tokenizer = AutoTokenizer.from_pretrained(
    CONFIG['base_model_id'],
    trust_remote_code=CONFIG['trust_remote_code']
)

load_kwargs = {
    'device_map': CONFIG['device_map'],
    'torch_dtype': CONFIG['dtype'],
    'trust_remote_code': CONFIG['trust_remote_code']
}

if CONFIG['load_in_8bit']:
    load_kwargs['load_in_8bit'] = True
elif CONFIG['load_in_4bit']:
    load_kwargs['load_in_4bit'] = True

base_model = AutoModelForCausalLM.from_pretrained(
    CONFIG['base_model_id'],
    **load_kwargs
)

# Aggiunta pad_token se mancante
if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token

print(f"Modello BASE caricato: {CONFIG['base_model_id']}")
print(f"Vocab size: {base_tokenizer.vocab_size}")


In [ ]:
# Caricamento del modello INSTRUCT
print("Caricamento modello INSTRUCT...")
instruct_tokenizer = AutoTokenizer.from_pretrained(
    CONFIG['instruct_model_id'],
    trust_remote_code=CONFIG['trust_remote_code']
)

instruct_model = AutoModelForCausalLM.from_pretrained(
    CONFIG['instruct_model_id'],
    **load_kwargs
)

if instruct_tokenizer.pad_token is None:
    instruct_tokenizer.pad_token = instruct_tokenizer.eos_token

print(f"Modello INSTRUCT caricato: {CONFIG['instruct_model_id']}")
print(f"Vocab size: {instruct_tokenizer.vocab_size}")
print(f"Chat template disponibile: {instruct_tokenizer.chat_template is not None}")


## Funzione di Generazione Unificata


In [ ]:
def generate(model, tokenizer, prompt_or_messages, is_chat=False):
    """
    Funzione unificata per generare testo con modelli BASE e INSTRUCT
    
    Args:
        model: Modello HuggingFace
        tokenizer: Tokenizer corrispondente
        prompt_or_messages: String (BASE) o lista di dict (INSTRUCT)
        is_chat: Se True, usa chat template (per INSTRUCT)
    
    Returns:
        dict con: text, num_input_tokens, num_output_tokens, latency_s
    """
    start_time = time.time()
    
    if is_chat and isinstance(prompt_or_messages, list):
        # Modalità chat (INSTRUCT)
        formatted_prompt = tokenizer.apply_chat_template(
            prompt_or_messages,
            add_generation_prompt=True,
            tokenize=False
        )
    else:
        # Modalità plain text (BASE)
        formatted_prompt = prompt_or_messages
    
    # Tokenizzazione
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model.device)
    
    num_input_tokens = inputs.input_ids.shape[1]
    
    # Generazione
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=CONFIG['max_new_tokens'],
            temperature=CONFIG['temperature'],
            do_sample=True if CONFIG['temperature'] > 0 else False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decodifica solo della parte generata
    generated_tokens = outputs[0][inputs.input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    num_output_tokens = len(generated_tokens)
    latency_s = time.time() - start_time
    
    return {
        'text': generated_text.strip(),
        'num_input_tokens': num_input_tokens,
        'num_output_tokens': num_output_tokens,
        'latency_s': latency_s
    }

print("Funzione di generazione definita!")


# SEZIONE 1: Confronto Prompting vs Chat Template

Mostriamo come lo stesso compito viene gestito diversamente dai due tipi di modello.


In [ ]:
# Task di esempio: spiegazione del colore del cielo
task_prompt = "Spiega in 3 punti perché il cielo è blu e termina con una frase conclusiva in grassetto."

print("=" * 60)
print("CONFRONTO: BASE vs INSTRUCT")
print("=" * 60)

# ============ MODELLO BASE - Zero-shot ============
print("\n🔸 MODELLO BASE (Zero-shot):")
base_prompt_zero = f"{task_prompt}\n\nRisposta:"
result_base_zero = generate(base_model, base_tokenizer, base_prompt_zero, is_chat=False)

print(f"Input tokens: {result_base_zero['num_input_tokens']}")
print(f"Output tokens: {result_base_zero['num_output_tokens']}")
print(f"Latency: {result_base_zero['latency_s']:.2f}s")
print(f"Risposta:\n{result_base_zero['text']}\n")


In [ ]:
# ============ MODELLO INSTRUCT - Chat Template ============
print("\n🔸 MODELLO INSTRUCT (Chat Template):")
instruct_messages = [
    {
        "role": "system", 
        "content": "Sei un assistente che risponde sempre in formato puntato numerato. Ogni risposta deve terminare con una frase conclusiva in grassetto."
    },
    {
        "role": "user", 
        "content": task_prompt
    }
]

result_instruct = generate(instruct_model, instruct_tokenizer, instruct_messages, is_chat=True)

print(f"Input tokens: {result_instruct['num_input_tokens']}")
print(f"Output tokens: {result_instruct['num_output_tokens']}")
print(f"Latency: {result_instruct['latency_s']:.2f}s")
print(f"Risposta:\n{result_instruct['text']}\n")

print("=" * 60)
print("OSSERVAZIONI SEZIONE 1:")
print("• Il modello BASE necessita di più contesto (few-shot) per seguire il formato")
print("• Il modello INSTRUCT segue immediatamente le istruzioni nel system prompt")
print("• La latency può variare significativamente tra i due approcci")
print("=" * 60)


# Note di Utilizzo

## Prossimi Passi
Il notebook ora è pronto per l'uso! Per estendere l'analisi, puoi aggiungere:

1. **Batteria di 5 Task di Valutazione**: QA, Sintesi, JSON, Aritmetica, Codice Python
2. **Test di Safety**: Confronto comportamenti di refusal
3. **Analisi Performance**: Visualizzazioni e metriche aggregate
4. **Fine-tuning Demo**: Setup LoRA per personalizzazione

## Requisiti Sistema
- **GPU**: Raccomandato almeno 16GB VRAM per i modelli 7B
- **RAM**: Minimo 32GB di RAM di sistema
- **Storage**: Circa 15GB per i modelli scaricati

## Configurazione Memoria
Se hai limitazioni di memoria, modifica la sezione CONFIG:
- `load_in_8bit: True` per ridurre l'uso di VRAM
- `load_in_4bit: True` per uso su GPU più piccole
- Riduci `max_new_tokens` per generazioni più veloci
